In [2]:
import duckdb as dk
import pandas as pd

con = dk.connect('data/vermont.duckdb')

In [30]:
con.close()

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [24]:
con.execute("""--sql
DROP TABLE IF EXISTS hotspot_vagrants;
CREATE TABLE hotspot_vagrants AS
SELECT
    locality_id,
    common_name,
    COUNT(DISTINCT EXTRACT(YEAR FROM observation_date)) AS years_observed
FROM sightings_clean
GROUP BY common_name, locality_id
;
""")

In [26]:
con.execute("""--sql
DROP TABLE IF EXISTS sightings_filtered;
CREATE TABLE sightings_filtered AS
SELECT
    s.*
FROM sightings_clean s
JOIN hotspot_vagrants h
    ON s.locality_id = h.locality_id
    AND s.common_name = h.common_name
    WHERE h.years_observed > 1
;
""")

In [29]:
con.execute("""--sql
SELECT
    ANY_VALUE(taxonomic_order) AS taxonomic_order,
    common_name
FROM sightings_filtered
WHERE locality == 'Shelburne Bay'
GROUP BY common_name
ORDER BY taxonomic_order
;
""").fetchdf()

,taxonomic_order,common_name
0,257,Snow Goose
1,276,Greater White-fronted Goose
2,302,Brant
3,319,Cackling Goose
4,330,Canada Goose
5,441,Wood Duck
6,468,Blue-winged Teal
7,483,Northern Shoveler
8,487,Gadwall
9,497,American Wigeon


In [12]:
con.execute("SELECT * FROM sightings_clean WHERE species_category = 'form'").fetchdf()

,global_id,last_edited_date,taxonomic_order,species_category,common_name,scientific_name,observation_count,country,country_code,state,state_code,county,county_code,locality,locality_id,locality_type,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_id,observation_type,duration_minutes,effort_distance_km,number_observers,all_species_reported,group_id,row_num
0,112894897,2018-01-04,32206,form,Redpoll,Acanthis flammea,14,United States,US,Vermont,US-VT,Washington,US-VT-023,North Branch Nature Center,166319,H,44.29,-72.57,2011-03-21,<NA>,135233,7884766,Incidental,<NA>,NaN,<NA>,True,240948,1
1,364834970,2024-04-25,5446,form,American Coot,Fulica americana,2,United States,US,Vermont,US-VT,Addison,US-VT-001,Long Point - Ferrisburgh,286684,H,44.25,-73.28,2016-01-09,11:19:00,223122,26804788,Traveling,58,0.32,3,True,1543893,1
2,699403523,2024-04-25,32206,form,Redpoll,Acanthis flammea,90,United States,US,Vermont,US-VT,Windsor,US-VT-027,Fox Stand (Royalton Hill Rd. bridge),1935490,H,43.82,-72.56,2019-01-17,08:45:00,139783,51763602,Traveling,75,1.21,2,True,3770264,1
3,1011677187,2025-07-22,32206,form,Redpoll,Acanthis flammea,1,United States,US,Vermont,US-VT,Chittenden,US-VT-007,Colchester Railroad Causeway,207196,H,44.57,-73.31,2020-11-14,08:17:00,1165223,76226897,Traveling,41,0.90,2,True,5905068,1
4,1044990131,2025-07-22,32206,form,Redpoll,Acanthis flammea,1,United States,US,Vermont,US-VT,Orleans,US-VT-019,Newport,165337,H,44.94,-72.21,2021-01-05,10:51:00,39970,78719372,Traveling,89,6.35,2,True,6147455,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593,1028734739,2025-07-22,32206,form,Redpoll,Acanthis flammea,16,United States,US,Vermont,US-VT,Rutland,US-VT-021,West Rutland Marsh IBA,109531,H,43.61,-73.06,2020-12-14,08:03:00,2437,77474597,Traveling,118,6.03,1,True,6003634,1
2594,1028632464,2024-04-25,32206,form,Redpoll,Acanthis flammea,1,United States,US,Vermont,US-VT,Windham,US-VT-025,Allen Brothers Marsh,201202,H,43.10,-72.45,2020-12-14,06:55:00,284151,77467068,Stationary,10,NaN,6,True,6004196,1
2595,1041087392,2025-07-22,32206,form,Redpoll,Acanthis flammea,12,United States,US,Vermont,US-VT,Franklin,US-VT-011,Missisquoi NWR--Stephen Young Marsh,272172,H,44.95,-73.21,2021-01-01,11:22:00,102014,78390841,Traveling,73,2.41,2,True,6081107,1
2596,1049035717,2021-02-24,32206,form,Redpoll,Acanthis flammea,100,United States,US,Vermont,US-VT,Chittenden,US-VT-007,Charlotte Town Beach,164833,H,44.33,-73.28,2021-01-10,14:15:00,901147,79049285,Traveling,45,0.32,2,True,6172614,1


In [23]:
con.execute("""--sql
SELECT
    *,
    "COMMON NAME" AS cn,
    "SCIENTIFIC NAME" AS sn,
    "LOCALITY" AS loc
FROM sightings_raw
WHERE contains(cn, 'Nelson') AND loc == 'Shelburne Bay'
;
""").fetchdf()

,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER,CATEGORY,TAXON CONCEPT ID,COMMON NAME,SCIENTIFIC NAME,SUBSPECIES COMMON NAME,SUBSPECIES SCIENTIFIC NAME,EXOTIC CODE,OBSERVATION COUNT,BREEDING CODE,BREEDING CATEGORY,BEHAVIOR CODE,AGE/SEX,COUNTRY,COUNTRY CODE,STATE,STATE CODE,COUNTY,COUNTY CODE,IBA CODE,BCR CODE,USFWS CODE,ATLAS BLOCK,LOCALITY,LOCALITY ID,LOCALITY TYPE,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,OBSERVER ORCID ID,SAMPLING EVENT IDENTIFIER,OBSERVATION TYPE,PROTOCOL NAME,PROTOCOL CODE,PROJECT NAMES,PROJECT IDENTIFIERS,DURATION MINUTES,EFFORT DISTANCE KM,EFFORT AREA HA,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,HAS MEDIA,APPROVED,REVIEWED,REASON,CHECKLIST COMMENTS,SPECIES COMMENTS,column52,cn,sn,loc
0,URN:CornellLabOfOrnithology:EBIRD:OBS42838141,2021-02-24 10:52:48.427803,32950,species,avibase-727D994F,Nelson's Sparrow,Ammospiza nelsoni,None,None,None,X,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Shelburne Bay,L165354,H,44.40,-73.23,2004-10-03,<NA>,obsr27544,None,S3050730,Incidental,Incidental,P20,None,<NA>,<NA>,NaN,NaN,<NA>,0,None,0,1,1,None,1st record since split accepted 11/6/04Theodor...,None,None,Nelson's Sparrow,Ammospiza nelsoni,Shelburne Bay
1,URN:CornellLabOfOrnithology:EBIRD:OBS42838144,2021-02-24 10:52:48.427803,32950,species,avibase-727D994F,Nelson's Sparrow,Ammospiza nelsoni,None,None,None,X,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Shelburne Bay,L165354,H,44.40,-73.23,2004-10-18,<NA>,obsr27544,None,S3050821,Incidental,Incidental,P20,None,<NA>,<NA>,NaN,NaN,<NA>,0,None,0,1,1,None,"accepted- 11/6/04R.B. Lavallee, Dennis Abbott",None,None,Nelson's Sparrow,Ammospiza nelsoni,Shelburne Bay
2,URN:CornellLabOfOrnithology:EBIRD:OBS42838142,2021-02-24 10:52:48.427803,32950,species,avibase-727D994F,Nelson's Sparrow,Ammospiza nelsoni,None,None,None,X,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Shelburne Bay,L165354,H,44.40,-73.23,2004-10-16,<NA>,obsr27544,None,S3050657,Incidental,Incidental,P20,None,<NA>,<NA>,NaN,NaN,<NA>,0,None,0,1,1,None,2nd record since split- accepted 11/6/04Frede...,None,None,Nelson's Sparrow,Ammospiza nelsoni,Shelburne Bay


In [ ]:
con.execute("SELECT * FROM rolling_avg_freq WHERE locality == 'Carse Wetlands (UVM Natural Area)' AND common_name =='Black-capped Chickadee'").fetchdf()